In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import swan_vis as swan


p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
## Adding "adult" metadata
sg = swan.read('swan.p')


Read in graph from swan.p


In [4]:
df = pd.read_csv('mouse_metadata.tsv', sep='\t')

In [6]:
df.age.unique()

array([nan, '10d', '14d', '18-20mo', '25d', '2mo', '36d', '4d', 'e11',
       'e13'], dtype=object)

In [17]:
df['adult'] = False

# tissues that aren't part of the b6cast timecourse are 
# adult
df.loc[(df.b6cast==False)&\
       (df.classification=='tissue')&\
       (df.tissue!='forelimb'), 'adult'] = True

# b6cast tissues w/ age = 18-20mo and 12mo are adult
ad_ages = ['18-20mo', '12mo']
df.loc[df.age.isin(ad_ages), 'adult'] = True

In [18]:
df.loc[df.adult==True, 'sample'].unique().tolist()

['adrenal',
 'adrenal_18-20mo',
 'cortex',
 'cortex_18-20mo',
 'gastroc_18-20mo',
 'heart_18-20mo',
 'hippocampus',
 'hippocampus_18-20mo']

In [24]:
# add column for tissue_adult
df['tissue_adult'] = df.tissue+'_'+df.adult.astype(str)
df.tissue_adult.unique()

array(['adrenal_True', 'adrenal_False', 'c2c12_myoblast_False',
       'c2c12_myotube_False', 'cortex_True', 'cortex_False',
       'f1219_False', 'forelimb_False', 'gastroc_False', 'gastroc_True',
       'heart_False', 'heart_True', 'hippocampus_True',
       'hippocampus_False'], dtype=object)

In [25]:
df.to_csv('mouse_metadata_adult.tsv', sep='\t', index=False)

In [26]:
sg.add_metadata('mouse_metadata_adult.tsv')

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:794: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [30]:
sg.save_graph('swan')

Saving graph as swan.p


In [ ]:
## something else idk 

In [13]:
df = pd.read_csv('../talon/mouse_talon_abundance_filtered.tsv', sep='\t')

df = get_mouse_metadata_from_ab(df)

# create sample level metadata
df['sample'] = False

# for timecourse+forelimb use tissue + age
inds = df.loc[(df.b6cast)|(df.forelimb)].index
df.loc[inds, 'sample'] = df.loc[inds, 'tissue']+'_'+df.loc[inds, 'age']

# for c2c12,  use tissue
inds = df.loc[df.c2c12==True].index
df.loc[inds, 'sample'] = df.loc[inds, 'tissue']

# for adrenal, hc, ctx from 5x, use tissue
inds = df.loc[(df['5x_v_wt']==True)|(df.adrenal==True)].index
df.loc[inds, 'sample'] = df.loc[inds, 'tissue']

df.to_csv('mouse_metadata.tsv', sep='\t', index=False)

123
123


In [22]:
df.sort_values(by=['dataset'], inplace=True)
df.to_csv('mouse_metadata.tsv', sep='\t', index=False)
print(df.columns)
print(df.head())
print(df.dtypes)

Index(['dataset', 'b6cast', 'tissue', 'age', 'sex', 'rep', '5x_v_wt',
       'disease_status', 'forelimb', 'c2c12', 'adrenal', 'sample'],
      dtype='object')
            dataset  b6cast   tissue  age sex  rep 5x_v_wt disease_status  \
6   adrenal_10d_f_1    True  adrenal  10d   f  1.0     NaN            NaN   
7   adrenal_10d_f_2    True  adrenal  10d   f  2.0     NaN            NaN   
36  adrenal_10d_m_1    True  adrenal  10d   m  1.0     NaN            NaN   
35  adrenal_10d_m_2    True  adrenal  10d   m  2.0     NaN            NaN   
42  adrenal_14d_f_1    True  adrenal  14d   f  1.0     NaN            NaN   

   forelimb c2c12 adrenal       sample  
6       NaN   NaN     NaN  adrenal_10d  
7       NaN   NaN     NaN  adrenal_10d  
36      NaN   NaN     NaN  adrenal_10d  
35      NaN   NaN     NaN  adrenal_10d  
42      NaN   NaN     NaN  adrenal_14d  
dataset            object
b6cast               bool
tissue             object
age                object
sex                object
r

In [23]:
old_df = pd.read_csv('/Users/fairliereese/mortazavi_lab/data/mousewg/lr_bulk/swan/mouse_metadata.tsv', sep='\t')
old_df.sort_values(by=['dataset'], inplace=True)
old_df.to_csv('mousewg_mouse_metadata.tsv', sep='\t', index=False)
print(old_df.columns)
print(old_df.head())
print(old_df.dtypes)
old_df.rename({'dataset':'beep'}, axis=1, inplace=True)

Index(['dataset', 'b6cast', 'tissue', 'age', 'sex', 'rep', '5x_v_wt',
       'disease_status', 'forelimb', 'c2c12', 'adrenal', 'sample'],
      dtype='object')
            dataset  b6cast   tissue  age sex  rep 5x_v_wt disease_status  \
6   adrenal_10d_f_1    True  adrenal  10d   f  1.0     NaN            NaN   
7   adrenal_10d_f_2    True  adrenal  10d   f  2.0     NaN            NaN   
36  adrenal_10d_m_1    True  adrenal  10d   m  1.0     NaN            NaN   
35  adrenal_10d_m_2    True  adrenal  10d   m  2.0     NaN            NaN   
42  adrenal_14d_f_1    True  adrenal  14d   f  1.0     NaN            NaN   

   forelimb c2c12 adrenal       sample  
6       NaN   NaN     NaN  adrenal_10d  
7       NaN   NaN     NaN  adrenal_10d  
36      NaN   NaN     NaN  adrenal_10d  
35      NaN   NaN     NaN  adrenal_10d  
42      NaN   NaN     NaN  adrenal_14d  
dataset            object
b6cast               bool
tissue             object
age                object
sex                object
r

In [24]:
pd.testing.assert_frame_equal(df, old_df, check_like=True, check_dtype=True)

AssertionError: DataFrame.columns are different

DataFrame.columns values are different (16.66667 %)
[left]:  Index(['5x_v_wt', 'adrenal', 'age', 'b6cast', 'c2c12', 'dataset',
       'disease_status', 'forelimb', 'rep', 'sample', 'sex', 'tissue'],
      dtype='object')
[right]: Index(['5x_v_wt', 'adrenal', 'age', 'b6cast', 'beep', 'c2c12',
       'disease_status', 'forelimb', 'rep', 'sample', 'sex', 'tissue'],
      dtype='object')